In [2]:
# 1. riss.kr 에서 특정 키워드로 논문 / 학술 자료 검색하기

#Step 1. 필요한 모듈을 로딩합니다
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time          

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print(" 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.")
print("=" *100)
query_txt = input('1.수집할 자료의 키워드는 무엇입니까? : ')

print("=" *100)
# input에 다중 문장을 넣을 때는 ''''''사용
print_2_que = '''2. 위 키워드로 아래의 장르 중 어떤 장르의 정보를 수집할까요?

1.학위논문\t2.국내학술논문\t3.해외학술논문\t4.학술지
5.단행본\t6.공개강의\t7.연구보고서

위 장르 중 수집할 장르의 번호를 입력하세요 : '''
print("=" *100)
query_number = int(input(print_2_que))

#Step 3. 수집된 데이터를 저장할 파일 이름 입력받기
fc_name = "C:\\Users\\Windows\\Desktop\\대학교\\4학년 여름방학\\Big_AI\\크롤링\\코드\\Chap_3\\출력 파일\\riss_problem1.csv"
fx_name = "C:\\Users\\Windows\\Desktop\\대학교\\4학년 여름방학\\Big_AI\\크롤링\\코드\\Chap_3\\출력 파일\\riss_problem1.xls"

#Step 4. 크롬 드라이버 설정 및 웹 페이지 열기
s = Service("C:\\chromedriver.exe")
driver = webdriver.Chrome(service=s)

url = 'https://www.riss.kr/'
driver.get(url)
time.sleep(3)
driver.maximize_window()

#Step 5. 자동으로 검색어 입력 후 조회하기
element = driver.find_element(By.ID,'query')
driver.find_element(By.ID,'query').click( )
element.send_keys(query_txt)
element.send_keys("\n")

#Step 6.입력 받은 장르 선택하기
input_genre = ""

if query_number == 1:
    input_genre = "학위논문" #  searchBox pd
elif query_number == 2:
    input_genre = "국내학술논문" # searchBox
elif query_number == 3:
    input_genre = "해외학술논문" # searchBox
elif query_number == 4:
    input_genre = "학술지" # searchBox pd
elif query_number == 5:
    input_genre = "단행본" # searchBox pd
elif query_number == 6:
    input_genre = "공개강의" # searchBox pd
elif query_number == 7:
    input_genre = "연구보고서" # searchBox pd

driver.find_element(By.LINK_TEXT,input_genre).click()
time.sleep(2)

#Step 7.Beautiful Soup 로 본문 내용만 추출하기
from bs4 import BeautifulSoup
html_1 = driver.page_source
soup_1 = BeautifulSoup(html_1, 'html.parser')

content_1 = soup_1.find('div','srchResultListW').find_all('li')
for i in content_1 :
    print(i.get_text().replace("\n",""))

#Step 8. 총 검색 건수를 보여주고 수집할 건수 입력받기
import math

if (query_number == 2) or (query_number == 3):
    total_cnt = soup_1.find('div','searchBox').find('span','num').get_text()
else:
    total_cnt = soup_1.find('div','searchBox pd').find('span','num').get_text()

print('검색하신 키워드 %s (으)로 총 %s 건의 학위논문이 검색되었습니다' %(query_txt,total_cnt))
collect_cnt = int(input('이 중에서 몇 건을 수집하시겠습니까?: '))
collect_page_cnt = math.ceil(collect_cnt / 10) # 올림
print('%s 건의 데이터를 수집하기 위해 %s 페이지의 게시물을 조회합니다.' %(collect_cnt,collect_page_cnt))
print('=' *80)

#Step 9. 각 항목별로 데이터를 추출하여 리스트에 저장하기
no2 = []        #번호 저장
title2 = []     #논문제목 저장
writer2 = []    #논문저자 저장
org2 = []       #소속기관 저장
year2 = []       #발표년도 저장
refer2 = []  #논문집/자료집 저장
URL2 = []         #논문 URL 저장

no = 1

for a in range(1, collect_page_cnt + 1) :
    html_2 = driver.page_source
    soup_2 = BeautifulSoup(html_2, 'html.parser')

    content_2 = soup_2.find('div','srchResultListW').find_all('li')
    
    for b in content_2 :    
        #1. 논문제목 있을 경우만
        try :
            title = b.find('div','cont').find('p','title').get_text()
        except :
            continue
        else :
            print('1.번호:',no)
            no2.append(no)

            print('2.논문제목:',title)
            title2.append(title)
            
            writer = b.find('span','writer').get_text()
            print('3.저자:',writer)
            writer2.append(writer)

            org = b.find('span','assigned').get_text()
            print('4.소속기관:' , org)
            org2.append(org)
            
            etc_value = b.find('p','etc').select("p > span")
            year = etc_value[2].get_text()
            print('5.발표년도:' , year)
            year2.append(year)
            
            refer = etc_value[3].get_text()
            print('6.논문집/자료집:' , refer)
            refer2.append(refer)
            
            # a tag에 href라는 속성값을 뽑아야 url을 뽑을 수 있다. soup.find('a')['href']
            URL = "http://www.riss.kr" + b.find('div','cont').find('p','title').find("a")["href"]
            print('7.논문 URL:' , URL)
            URL2.append(URL)
            
            no += 1
            print("\n")
            
            if no > collect_cnt :
                break

            time.sleep(1)        # 페이지 변경 전 1초 대기 

    a += 1 
    b = str(a)

    try :
        driver.find_element(By.LINK_TEXT ,'%s' %b).click() 
    except :
        driver.find_element(By.LINK_TEXT, '다음 페이지로').click()
        
print("요청하신 작업이 모두 완료되었습니다")

# Step 10. 수집된 데이터를 xls와 csv 형태로 저장하기
import pandas as pd 

df = pd.DataFrame()
df['번호']=no2
df['제목']=pd.Series(title2) # 결측값이 있으면 df["제목"] = title2가 작동 x
df['저자']=pd.Series(writer2)
df['소속(발행)기관']=pd.Series(org2)
df["발표년도"] = pd.Series(year2)
df["논문집/자료집"] = pd.Series(refer2)
df["논문 URL"] = pd.Series(URL2)

# xls 형태로 저장하기
df.to_excel(fx_name,index=False, encoding="utf-8" , engine='openpyxl')

# csv 형태로 저장하기
df.to_csv(fc_name,index=False, encoding="utf-8-sig")

print('요청하신 데이터 수집 작업이 정상적으로 완료되었습니다')

 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.
1.수집할 자료의 키워드는 무엇입니까? : 해양자원
2. 위 키워드로 아래의 장르 중 어떤 장르의 정보를 수집할까요?

1.학위논문	2.국내학술논문	3.해외학술논문	4.학술지
5.단행본	6.공개강의	7.연구보고서

위 장르 중 수집할 장르의 번호를 입력하세요 : 2
1발표논문 / 해양생물자원으로서 해조류 : 생물활성물질의 정제와 분자적 응용홍용기(Yong Ki Hong)한국조류학회2000국제심포지움 일정 및 발표논문집 - 21세기, 해양환경과 해양생물자원의 전망Vol.- No.-해조생물공학은 유용해조류 및 그들의 유용물질들을 경제적으로 생산할 수 있도록 개발하는 학문 분야이다. 본 논문에서는 이들 해조생물공학에 대한 실험방법적 접근을 강조하여 설명하고자 한다. 우선 해조류의 활력측정은 조직 자체의 효소활성 즉 무색의 2,3,5-triphenyltetrazolium chloride를 붉은색의 triphenylformazan으로 환원시키는 정도를 측정함으로서 정량적으로 간단하게 활력을 평가하는 방법을 확립하였다. 그리고 callus와 엽체 재생 등을 위한 무균 고체 조직배양에 가장 적합한 gelling agent들을 비교한바 high gel strength agar가 톳의 callus 형성 및 엽체 재생에 지지체로 가장 효율적이였다. 유용 생리활성물질의 탐색작업에서는 보라색우무 추출물에서 Taq DNA polymerase에 대한 저해작용을 보였으며, 청각, 구멍갈파래, 잎파래, 미역에는 Herpes simplex virus에 대한 항virus효과를 지녔다. 홑파래물추출물에서는 생먹이용 미세조류들의 성장에 촉진효과를 보였으며, 작은구슬산호말은 적조들의 성장을 억제시켰다. 넓패는 가시파래 및 담치에 대한 부착 억제작용을 보였다. 또한 김과 홑파래는 고cholesterol혈증에서 cholesterol수치를 강하시켰다.  그리고 해조류들의 유전분석을 위하여 DNA와 RNA 추출은 LiCl를 사용하여

이 중에서 몇 건을 수집하시겠습니까?: 3
3 건의 데이터를 수집하기 위해 1 페이지의 게시물을 조회합니다.
1.번호: 1
2.논문제목: 발표논문 / 해양생물자원으로서 해조류 : 생물활성물질의 정제와 분자적 응용
3.저자: 홍용기(Yong Ki Hong)
4.소속기관: 한국조류학회
5.발표년도: 2000
6.논문집/자료집: 국제심포지움 일정 및 발표논문집 - 21세기, 해양환경과 해양생물자원의 전망
7.논문 URL: http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=508c47ca1906d5ecffe0bdc3ef48d419&keyword=해양자원


1.번호: 2
2.논문제목: 수거된 해양폐기물 자원화 기술 개발(Ⅰ)
3.저자: 길상인(Sang-In Keel),윤진한(Jin-Han Yun),최연석(Yeon-Seok Choi),강창구(Chang-Gu Kang),유정석(Jeong-Seok Yu)
4.소속기관: 한국해양환경·에너지학회
5.발표년도: 2002
6.논문집/자료집: 한국해양환경·에너지학회지
7.논문 URL: http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=09ec1e3541fcc118ffe0bdc3ef48d419&keyword=해양자원


1.번호: 3
2.논문제목: 수거된 해양폐기물 자원화 기술 개발(Ⅱ)
3.저자: 길상인(Sang-In Keel),김석준(Seock-Joon Kim),윤진한(Jin-Han Yun),강창구(Chang-Gu Kang),유정석(Jeong-Seok Yu)
4.소속기관: 한국해양환경·에너지학회
5.발표년도: 2002
6.논문집/자료집: 한국해양환경·에너지학회지
7.논문 URL: http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_

In [ ]:
# 2. 네이버에서 특정 키워드로 논문 / 학술 자료 검색하기

#Step 1. 필요한 모듈을 로딩합니다
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time          

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print("이 크롤러는 네이버 블로그 수집용 웹크롤러입니다.")
print("=" *100)
query_txt = input('1. 크롤링할 키워드는 무엇입니까? : ')
collect_cnt = int(input('2. 수집할 데이터는 몇 건입니까? : '))
print("=" *100)

#Step 3. 수집된 데이터를 저장할 파일 이름 입력받기
fc_name = "C:\\Users\\Windows\\Desktop\\대학교\\4학년 여름방학\\Big_AI\\크롤링\\코드\\Chap_3\\출력 파일\\naver_problem2.csv"
fx_name = "C:\\Users\\Windows\\Desktop\\대학교\\4학년 여름방학\\Big_AI\\크롤링\\코드\\Chap_3\\출력 파일\\naver_problem2.xls"

#Step 4. 크롬 드라이버 설정 및 웹 페이지 열기
s = Service("C:\\chromedriver.exe")
driver = webdriver.Chrome(service=s)

url = 'https://www.naver.com/'
driver.get(url)
time.sleep(3)
driver.maximize_window()

#Step 5. 자동으로 검색어 입력 후 조회하기
element = driver.find_element(By.ID,'query')
driver.find_element(By.ID,'query').click( )
element.send_keys(query_txt)
element.send_keys("\n")

#Step 6.View-Blog 선택하기
driver.find_element(By.LINK_TEXT,"VIEW").click()
time.sleep(1)
driver.find_element(By.LINK_TEXT,"블로그").click()
time.sleep(1)

#Step 7.Beautiful Soup 로 본문 내용만 추출하기
from bs4 import BeautifulSoup
html_1 = driver.page_source
soup_1 = BeautifulSoup(html_1, 'html.parser')

content_1 = soup_1.find('div','_more_contents_event_base').find_all('li')
for i in content_1:
    print(i.get_text().replace("\n",""))

#Step 8. 각 항목별로 데이터를 추출하여 리스트에 저장하기
no2 = []        #번호 저장
title2 = []     #제목 저장
mean2 = []    #요약내용 저장
day2 = []       #작성일자 저장
nickname2 = []       #블로그 닉네임 저장

no = 1

for i in content_1:   
    try :
        title = i.find('div','total_area').find('a','api_txt_lines total_tit').get_text()
    except :
        continue
    else :
            print('1.번호:',no)
            no2.append(no)

            print('2.제목:',title)
            title2.append(title)
            
            mean = i.find('div','total_area').find('a','total_dsc').get_text()
            print('3.요약내용:',mean)
            mean2.append(mean)

            day = i.find('div','total_area').find('span', 'sub_time sub_txt').get_text()
            print('4.작성일자:' , day)
            day2.append(day)
            
            nickname = i.find('div','total_area').find('a', 'sub_txt sub_name').get_text()
            print('5.블로그닉네임:' , nickname)
            nickname2.append(nickname)
            
            no += 1
            print("\n")
            
            if no > collect_cnt :
                break
        
print("요청하신 작업이 모두 완료되었습니다")

# Step 10. 수집된 데이터를 xls와 csv 형태로 저장하기
import pandas as pd 

df = pd.DataFrame()
df['번호']=no2
df['제목']=pd.Series(title2)
df['요약내용']=pd.Series(mean2)
df['작성일자']=pd.Series(day2)
df["블로그닉네임"] = pd.Series(nickname2)

# xls 형태로 저장하기
df.to_excel(fx_name,index=False, encoding="utf-8" , engine='openpyxl')

# csv 형태로 저장하기
df.to_csv(fc_name,index=False, encoding="utf-8-sig")

print('요청하신 데이터 수집 작업이 정상적으로 완료되었습니다')

이 크롤러는 네이버 블로그 수집용 웹크롤러입니다.
1. 크롤링할 키워드는 무엇입니까? : 서진수 빅데이터
2. 수집할 데이터는 몇 건입니까? : 15
